In [ ]:
from sklearn.metrics import mean_absolute_error as MSE
from tensorflow.keras.models import Sequential
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

# Loading our data

In [ ]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")

price = df['NO2_price'].values.reshape(-1,1)
fload = df['NO2_load_forecasted'].values.reshape(-1,1)
fgen = df['NO2_generation_forecast'].values.reshape(-1,1)

price_days = seperate_column_to_days(price)
fload_days = seperate_column_to_days(fload)
fgen_days = seperate_column_to_days(fgen)

# Structuring our data

In [ ]:
lookbehind = 4
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
stride = 24
hour_in_days = int(no_hours / stride)

price_dataset = []
fload_dataset = []
fgen_dataset = []

for i in range(len(price_days) - hour_in_days+1):
    price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))
    fload_dataset.append(np.concatenate((fload_days[i:i+hour_in_days])))
    fgen_dataset.append(np.concatenate((fgen_days[i:i+hour_in_days])))

price_dataset = np.array(price_dataset)
fload_dataset = np.array(fload_dataset)
fgen_dataset = np.array(fgen_dataset)

scaler = MinMaxScaler()
price_dataset = scaler.fit_transform(price_dataset[:,:,0])
price_dataset = price_dataset[..., np.newaxis].astype(np.float32)

fload_dataset = scaler.fit_transform(fload_dataset[:,:,0])
fload_dataset = fload_dataset[..., np.newaxis].astype(np.float32)

fgen_dataset = scaler.fit_transform(fgen_dataset[:,:,0])
fgen_dataset = fgen_dataset[..., np.newaxis].astype(np.float32)

dataset_forecast = np.concatenate((price_dataset, fload_dataset, fgen_dataset), axis=2)

n,m,k = dataset_forecast.shape

print(dataset_forecast.shape)


# Train test split

In [ ]:
train = int(0.7*n)
valid = int(0.9*n)
x_train, X_train = price_dataset[:train, :input_width], dataset_forecast[:train, :input_width]
x_valid, X_valid = price_dataset[train:valid, :input_width], dataset_forecast[train:valid, :input_width]
x_test, X_test = price_dataset[valid:, :input_width], dataset_forecast[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset_forecast[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

In [ ]:
print(X_train.shape)

## Hyperparameters

In [ ]:
# Common
epochs = 1
batch_size = [200, 300]
batch_size = [200]
eta_list = np.logspace(-2, -4, 3)
neurons_list = [128,64, 32]
neurons_list = [128]
filters = 64
kernel_size = 3
strides = 3

learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            patience=3, 
                                                            verbose=1, 
                                                            factor=0.5, 
                                                            min_lr=0.000001)
callbacks = [learning_rate_reduction]

# Setting up the univariate SimpleRNN

In [ ]:
class RNN(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()

        # RNN architecture
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True, input_shape=[None, k]))
        self.add(keras.layers.SimpleRNN(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)


# return_sequences=True --> (256, 168, 24)
# return_sequences=False --> (256, 24)

## Grid search

In [ ]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = Y_test[:,-1]
best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            RNN_model = RNN(eta, neurons, horizon=horizon)

            # Training
            history = RNN_model.fit(X_train, Y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = RNN_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}multivariate_simpleRNN_results_control.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/multivariate_simpleRNN_preds_best_control.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/multivariate_simpleRNN_results_best_control.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/multivariate_simpleRNN_train_val_losses_best_control.csv")            
                
            i += 1

# Setting up the multivariate ConvGRU

In [ ]:
class ConvGRU(Sequential):
    def __init__(self, eta, neurons_first, neurons_second, horizon=24):
        super(self.__class__, self).__init__()

        # ConvGRU architecture
        self.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]))
        self.add(keras.layers.GRU(neurons_first, return_sequences=True))
        self.add(keras.layers.GRU(neurons_second, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [ ]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]
Y_test_last = Y_test[:,-1]
best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            ConvGRU_model = ConvGRU(eta, neurons, neurons, horizon=horizon)

            # Training
            history = ConvGRU_model.fit(X_train, Y_train[:,2::strides], 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid[:,2::strides]), verbose=0)

            # Prediction
            Y_pred_rnn = ConvGRU_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
                        
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}multivariate_convGRU_results_control.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/multivariate_convGRU_preds_best_control.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/multivariate_convGRU_results_best_control.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/multivariate_convGRU_train_val_losses_best_control.csv")            
                
            i += 1

# Setting up the multivariate LSTM

In [ ]:
class LSTM(Sequential):
    def __init__(self, eta, neurons, horizon=24):
        super(self.__class__, self).__init__()
        
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        # self.add(keras.layers.Bidirectional(keras.layers.LSTM(neurons, return_sequences=True)))
        
        self.add(keras.layers.LSTM(neurons, return_sequences=True))
        self.add(keras.layers.LSTM(neurons, return_sequences=True))
        self.add(keras.layers.Dense(horizon))

        # Model compile settings
        opt = keras.optimizers.Adam(learning_rate=eta)

        # Compile model
        self.compile(loss='mse', optimizer=opt)

## Grid search

In [ ]:
results_columns = ["model","batch_size", "learning_rate", "epoch", "neurons", "mse_all"]
results_columns += [f"mse{i}" for i in range(1,horizon+1)] 
results = pd.DataFrame(columns=results_columns)

t_columns = [f"t{k}" for k in range(1, horizon+1)]
t_hat_columns = [f"t_hat{k}" for k in range(1, horizon+1)]
preds_columns = columns=t_columns+t_hat_columns
results_preds = pd.DataFrame(columns=preds_columns)

epochs_colmns = [f"epoch{i}" for i in range(epochs)]

Y_test_last = Y_test[:,-1]

best_mse = np.inf

i = 0
for eta in tqdm(eta_list):
    for batch in batch_size:
        for neurons in neurons_list:
            LSTM_model = LSTM(eta, neurons, horizon=horizon)

            # Training
            history = LSTM_model.fit(X_train, Y_train, 
                        epochs=epochs, batch_size = batch, 
                        shuffle=False, callbacks= callbacks,  
                        validation_data=(X_valid, Y_valid), verbose=0)

            # Prediction
            Y_pred_rnn = LSTM_model.predict(X_test)
            Y_pred_rnn_last = Y_pred_rnn[:,-1]     
            
            # Results
            mse_val = MSE(Y_test_last, Y_pred_rnn_last)
            mse_all = np.mean((Y_test_last - Y_pred_rnn_last) ** 2, axis=0) # The mean of each column 
            
            
            # Storing results for all models
            res1 = np.array([i, batch, eta, epochs, neurons, mse_val])
            res2 = np.array(mse_all)
            res = np.concatenate([res1,res2], axis=0)
            results.loc[i] = res
            results.to_csv(f"{REPORT_DATA}multivariate_LSTM_results_control.csv")

            # Storing results from the best model
            if best_mse > mse_val:
                best_mse = mse_val
                print(f"\nNew best model with MSE = {mse_val} !\n")
                # Predictions and target for best results
                preds_res =  np.concatenate([Y_test_last, Y_pred_rnn_last],axis=1)
                best_results_preds = pd.DataFrame(preds_res, columns=preds_columns)
                best_results_preds.to_csv(f"{REPORT_DATA}best_models/multivariate_LSTM_preds_best_control.csv")

                
                # Best results relative to parameters
                best_results = pd.DataFrame(columns=results_columns)
                best_results.loc[i] = res
                best_results.to_csv(f"{REPORT_DATA}best_models/multivariate_LSTM_results_best_control.csv")
                
                # Model evaluation result            
                best_train_val_losses = pd.DataFrame(columns=["training_loss", "validation_loss"])
                best_train_val_losses["training_loss"] = history.history["loss"]
                best_train_val_losses["validation_loss"] = history.history["val_loss"]
                best_train_val_losses.to_csv(f"{REPORT_DATA}best_models/multivariate_LSTM_train_val_losses_best_control.csv")            
                
            i += 1